In [2]:
import pandas as pd
import numpy as np
import copy
import matplotlib.pyplot as plt
import itertools
from tqdm import tqdm
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from datetime import timedelta
import pmdarima as pm
from pmdarima.arima import auto_arima

### 데이터 전처리 

In [3]:
edge_daegu_infect = np.load('./data/raw_data/edge_daegu_infect.npy', allow_pickle=True)
edge_daegu_user_1st = np.load('./data/raw_data/edge_daegu_user_1st.npy', allow_pickle=True)
edge_daegu_user_2nd = np.load('./data/raw_data/edge_daegu_user_2nd.npy', allow_pickle=True)
edge_daegu_user_3rd = np.load('./data/raw_data/edge_daegu_user_3rd.npy', allow_pickle=True)
edge_daegu_user_4th = np.load('./data/raw_data/edge_daegu_user_4th.npy', allow_pickle=True)
edge_daegu_user_5th = np.load('./data/raw_data/edge_daegu_user_5th.npy', allow_pickle=True)
node_daegu_infect = np.load('./data/raw_data/node_daegu_infect.npy', allow_pickle=True)
node_daegu_user = np.load('./data/raw_data/node_daegu_user.npy', allow_pickle=True)
edge_daegu_user = np.concatenate((edge_daegu_user_1st,edge_daegu_user_2nd,edge_daegu_user_3rd,edge_daegu_user_4th,edge_daegu_user_5th))
del (edge_daegu_user_1st, edge_daegu_user_2nd, edge_daegu_user_3rd, edge_daegu_user_4th, edge_daegu_user_5th)

In [4]:
node_user = (node_daegu_user.reshape([-1, 142])-np.min(node_daegu_user))/(np.max(node_daegu_user)-np.min(node_daegu_user))
node_infect = (node_daegu_infect.reshape([-1, 142])-np.min(node_daegu_infect))/(np.max(node_daegu_infect)-np.min(node_daegu_infect))
edge_user = np.array([edge_daegu_user[:,0]+edge_daegu_user[:,1], edge_daegu_user[:,2], edge_daegu_user[:,3],
                      (edge_daegu_user[:,4]-np.min(edge_daegu_user[:,4]))/(np.max(edge_daegu_user[:,4])-np.min(edge_daegu_user[:,4]))]).transpose()
edge_infect = np.array([edge_daegu_infect[:,0]+edge_daegu_infect[:,1], edge_daegu_infect[:,2], edge_daegu_infect[:,3],
                        (edge_daegu_infect[:,4]-np.min(edge_daegu_infect[:,4]))/(np.max(edge_daegu_infect[:,4])-np.min(edge_daegu_infect[:,4]))]).transpose()

ymd_time_lst = np.sort(np.unique(edge_user[:,0]))
dong_lst = np.sort(np.unique(edge_user[:,1])) 

In [5]:
dong_sty_lst = np.array([dong for i in range(696) for dong in dong_lst])
ymd_time_sty_lst = np.array([ymd_time for ymd_time in ymd_time_lst for i in range(142)])
sty_infect_ts = pd.DataFrame(np.concatenate((ymd_time_sty_lst.reshape(98832,1), dong_sty_lst.reshape(98832,1)), axis=1))
sty_infect_ts['sty_num'] = node_infect.reshape(98832)
sty_infect_ts.columns = ['ymd_time', 'dong','sty_num']

### mv

In [6]:
raw_mv_infect = pd.DataFrame(edge_infect, columns=['ymd_time', 'from_dong', 'to_dong', 'mv_num'])
mv_infect = raw_mv_infect.loc[1:]
to_dong_grouped = mv_infect.groupby(['ymd_time', 'to_dong']).sum()['mv_num']
to_dong_grouped = to_dong_grouped.reset_index()
to_dong_mv = np.zeros([696, 142])
for i, ymd_time in tqdm(enumerate(ymd_time_lst)):
    for j, dong in enumerate(dong_lst):
        if not to_dong_grouped[(to_dong_grouped['ymd_time']==ymd_time) & (to_dong_grouped['to_dong']==dong)].empty:
            to_dong_mv[i][j] = to_dong_grouped[(to_dong_grouped['ymd_time']==ymd_time) & (to_dong_grouped['to_dong']==dong)]['mv_num'].item()

0it [00:00, ?it/s]/home/hyungjun/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: FutureWarning: `item` has been deprecated and will be removed in a future version
  if __name__ == '__main__':
696it [31:18,  2.70s/it]


In [7]:
dong_sty_lst = np.array([dong for i in range(696) for dong in dong_lst])
ymd_time_sty_lst = np.array([ymd_time for ymd_time in ymd_time_lst for i in range(142)])
mv_infect_ts = pd.DataFrame(np.concatenate((ymd_time_sty_lst.reshape(98832,1), dong_sty_lst.reshape(98832,1)), axis=1))
mv_infect_ts['mv_num'] = to_dong_mv.reshape(98832)
mv_infect_ts.columns = ['ymd_time', 'dong','mv_num']

In [8]:
sty_infect_ts_lst = []
for i in range(len(dong_lst)):
    sty_infect_ts_lst.append(sty_infect_ts[sty_infect_ts['dong']==dong_lst[i]])

In [9]:
mv_infect_ts_lst = []
for i in range(len(dong_lst)):
    mv_infect_ts_lst.append(mv_infect_ts[mv_infect_ts['dong']==dong_lst[i]])

# ARIMA

### 필요 함수

In [10]:
def forecast_fnc(model, n_period):
    fc, conf_int = model.predict(n_periods=n_period, return_conf_int=True)
    return fc.tolist(), np.asarray(conf_int).tolist()

def accuracy(forecast, true):
    return mse(forecast, true), mae(forecast, true)

In [ ]:
TEST_DAYS=7
forecasts = np.zeros([142, 24*TEST_DAYS])
dong_target = np.zeros([142, 24*TEST_DAYS])

for i, (sty_infect_ts_dong, mv_infect_ts_dong) in tqdm(enumerate(zip(sty_infect_ts_lst, mv_infect_ts_lst))):
    print('='*20, i, '='*20)
    sty_dong_train = sty_infect_ts_dong["sty_num"][0:(696-24*TEST_DAYS)]
    sty_dong_test = sty_infect_ts_dong["sty_num"][(696-24*TEST_DAYS):]

    mv_dong_train = mv_infect_ts_dong["mv_num"][0:(696-24*TEST_DAYS)]
    mv_dong_test = mv_infect_ts_dong["mv_num"][(696-24*TEST_DAYS):]

    dong_train = sty_dong_train + mv_dong_train
    dong_test = sty_dong_test + mv_dong_test
    dong_target[i] = dong_test
    
    model = pm.auto_arima(dong_train, start_p=0, start_q=0,
                          max_p=3, max_q=3, m=1,
                          start_P=0, seasonal=False,
                          d=1, D=0, trace=True,
                          error_action='ignore',  # don't want to know if an order does not work
                          suppress_warnings=True,  # don't want convergence warnings
                          stepwise=True)  # set to stepwise
    train_predict = model.predict_in_sample(dong_train)
    forecast_dong, _ = forecast_fnc(model, 24*TEST_DAYS)
    forecasts[i] = np.array(forecast_dong)
    

In [ ]:
accuracys = []
for i in range(24*TEST_DAYS):
    accuracy.append(accuracy(forecasts[:, i], dong_target[:, i]))


In [26]:
np.mean(mse_lst)

0.41075441701144055

In [27]:
np.mean(mae_lst)

0.29543139578478783

# simple DNN